In [1]:
import scipy
import os

In [2]:
from pathlib import Path
import numpy as np
from scipy.cluster.vq import whiten, kmeans, vq

In [3]:
data = Path("SMSSpamCollection").read_text()
data = data.strip()
data = data.split("\n")

In [4]:
data[2]

"spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [5]:
# np empty creates spicified dimension array with random number, making it faster than zeroes.
digit_counts = np.empty((len(data), 2), dtype=int)
digit_counts

array([[           43609960,             9445568],
       [             477907,                  -1],
       [8439868052583833960, 8247614980089279598],
       ...,
       [3203312157427004021, 7594230491502309989],
       [2538750822798339187, 7496855245682927398],
       [2338038244119768417, 6998709460502996296]])

In [6]:
for i, line in enumerate(data):
    case, message = line.split("\t")
    num_digits = sum(c.isdigit() for c in message)
    digit_counts[i, 0] = 0 if case == "ham" else 1
    digit_counts[i, 1] = num_digits

In [7]:
digit_counts

array([[ 0,  0],
       [ 0,  0],
       [ 1, 25],
       ...,
       [ 0,  0],
       [ 0,  0],
       [ 0,  0]])

In [8]:
unique_counts = np.unique(digit_counts[:, 1], return_counts=True)
unique_counts

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 40, 41, 47]),
 array([4110,  486,  160,   78,   42,   39,   16,   14,   28,   17,   16,
          34,   30,   31,   37,   29,   35,   33,   41,   47,   18,   31,
          28,   36,   34,   16,   16,   13,   19,    9,    2,    6,    3,
           4,    3,    4,    1,    1,    4,    2,    1]))

In [9]:
np.vstack(unique_counts)

array([[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
          22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
          33,   34,   35,   36,   37,   40,   41,   47],
       [4110,  486,  160,   78,   42,   39,   16,   14,   28,   17,   16,
          34,   30,   31,   37,   29,   35,   33,   41,   47,   18,   31,
          28,   36,   34,   16,   16,   13,   19,    9,    2,    6,    3,
           4,    3,    4,    1,    1,    4,    2,    1]])

In [10]:
unique_counts = np.transpose(np.vstack(unique_counts))
unique_counts

array([[   0, 4110],
       [   1,  486],
       [   2,  160],
       [   3,   78],
       [   4,   42],
       [   5,   39],
       [   6,   16],
       [   7,   14],
       [   8,   28],
       [   9,   17],
       [  10,   16],
       [  11,   34],
       [  12,   30],
       [  13,   31],
       [  14,   37],
       [  15,   29],
       [  16,   35],
       [  17,   33],
       [  18,   41],
       [  19,   47],
       [  20,   18],
       [  21,   31],
       [  22,   28],
       [  23,   36],
       [  24,   34],
       [  25,   16],
       [  26,   16],
       [  27,   13],
       [  28,   19],
       [  29,    9],
       [  30,    2],
       [  31,    6],
       [  32,    3],
       [  33,    4],
       [  34,    3],
       [  35,    4],
       [  36,    1],
       [  37,    1],
       [  40,    4],
       [  41,    2],
       [  47,    1]])

# Kmeans

In [11]:
# codebook is an array containing points for each centroid
# x is the mean euclidean distance of observation from centroid
whitened_counts = whiten(unique_counts)
codebook, x = kmeans(whitened_counts, 3)

In [12]:
codebook

array([[2.52050073, 0.01840656],
       [0.85234324, 0.09724666],
       [0.        , 6.49364346]])

In [13]:
x

0.42816013482347576

In [14]:
# codes is the cluster each point in unique_counts belongs to
# _ is the euclidian distance each point is from its centroid
codes, _ = vq(whitened_counts, codebook)

In [18]:
codes

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [19]:
codes[-1]

0

In [16]:
_

array([0.        , 1.02197058, 0.70730765, 0.60937111, 0.52854438,
       0.44788482, 0.37231175, 0.29387939, 0.20974747, 0.1406438 ,
       0.08262357, 0.05951512, 0.13157169, 0.20860002, 0.28674992,
       0.36889236, 0.4484318 , 0.52956568, 0.60968175, 0.69037499,
       0.77423127, 0.81638688, 0.73509273, 0.65459468, 0.57337601,
       0.49116015, 0.40999412, 0.32876788, 0.24785761, 0.16646253,
       0.08658724, 0.00980618, 0.07831841, 0.1587531 , 0.23985753,
       0.32087113, 0.40217112, 0.48328752, 0.72662168, 0.80784058,
       1.29485931])

In [20]:
ham_code = codes[0]
spam_code = codes[-1]
unknown_code = list(set(range(3)) ^ set((ham_code, spam_code)))[0]

In [21]:
ham_code

2

In [22]:
spam_code

0

In [23]:
unknown_code

1

In [24]:
print("definitely ham:", unique_counts[codes == ham_code][-1])
print("definitely spam:", unique_counts[codes == spam_code][-1])
print("unknown:", unique_counts[codes == unknown_code][-1])

definitely ham: [   0 4110]
definitely spam: [47  1]
unknown: [20 18]


In [27]:
digits = digit_counts[:, 1]
predicted_hams = digits == 0
predicted_spams = digits > 20
predicted_unknowns = np.logical_and(digits > 0, digits <= 20)

In [28]:
digits

array([ 0,  0, 25, ...,  0,  0,  0])

In [29]:
predicted_hams

array([ True,  True, False, ...,  True,  True,  True])

In [31]:
predicted_spams

array([False, False,  True, ..., False, False, False])

In [30]:
predicted_unknowns

array([False, False, False, ..., False, False, False])

In [32]:
spam_cluster = digit_counts[predicted_spams]
ham_cluster = digit_counts[predicted_hams]
unk_cluster = digit_counts[predicted_unknowns]

In [33]:
spam_cluster

array([[ 1, 25],
       [ 1, 22],
       [ 1, 23],
       [ 1, 22],
       [ 1, 24],
       [ 1, 21],
       [ 1, 22],
       [ 1, 22],
       [ 1, 22],
       [ 1, 34],
       [ 1, 26],
       [ 1, 22],
       [ 1, 22],
       [ 1, 31],
       [ 1, 24],
       [ 1, 22],
       [ 1, 23],
       [ 1, 22],
       [ 1, 28],
       [ 1, 28],
       [ 1, 31],
       [ 1, 21],
       [ 1, 33],
       [ 1, 24],
       [ 1, 24],
       [ 1, 41],
       [ 1, 24],
       [ 1, 24],
       [ 1, 40],
       [ 1, 22],
       [ 1, 24],
       [ 1, 40],
       [ 1, 27],
       [ 1, 21],
       [ 1, 29],
       [ 1, 24],
       [ 1, 21],
       [ 1, 21],
       [ 1, 23],
       [ 1, 47],
       [ 1, 22],
       [ 1, 33],
       [ 1, 25],
       [ 1, 30],
       [ 1, 29],
       [ 1, 25],
       [ 1, 25],
       [ 1, 27],
       [ 1, 24],
       [ 1, 34],
       [ 1, 28],
       [ 1, 28],
       [ 1, 24],
       [ 1, 28],
       [ 1, 28],
       [ 0, 23],
       [ 1, 22],
       [ 1, 22],
       [ 1, 28

In [35]:
ham_cluster

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [36]:
unk_cluster

array([[ 1,  4],
       [ 0,  1],
       [ 1, 19],
       ...,
       [ 0,  1],
       [ 0,  1],
       [ 1,  5]])

In [37]:
print("hams:", np.unique(ham_cluster[:, 0], return_counts=True))
print("spams:", np.unique(spam_cluster[:, 0], return_counts=True))
print("unknowns:", np.unique(unk_cluster[:, 0], return_counts=True))

hams: (array([0, 1]), array([4071,   39]))
spams: (array([0, 1]), array([  1, 232]))
unknowns: (array([0, 1]), array([755, 476]))


# Optimizing 

In [39]:
from scipy.optimize import minimize_scalar

In [41]:
def objective_function(x):
    return 3 * x ** 4 - 2 * x + 1

In [43]:
res = minimize_scalar(objective_function)

In [ ]:
#brent is an implementation of Brent’s algorithm. This method is the default.
#golden is an implementation of the golden-section search. The documentation notes that Brent’s method is usually better.
#bounded is a bounded implementation of Brent’s algorithm. It’s useful to limit the search region when the minimum is in a known range.

In [44]:
# fun: value of y at minimum
# x: value of x at minimum
res

     fun: 0.17451818777634331
    nfev: 16
     nit: 12
 success: True
       x: 0.5503212087491959

In [47]:
def objective_function(x):
    return x ** 4 - x ** 2

In [52]:
res = minimize_scalar(objective_function)

In [53]:
res

     fun: -0.24999999999999994
    nfev: 15
     nit: 11
 success: True
       x: 0.7071067853059209

In [56]:
res = minimize_scalar(objective_function, bracket=(-1, 0))

In [57]:
res

     fun: -0.24999999999999997
    nfev: 17
     nit: 13
 success: True
       x: 0.7071067809244586

In [58]:
res = minimize_scalar(objective_function, method='bounded', bounds=(-1, 0))

In [59]:
res

     fun: -0.24999999999998732
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: -0.707106701474177

# Minimizing function with multiple variable

In [61]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

n_buyers = 10
n_shares = 15

In [ ]:
#LinearConstraint: The solution is constrained by taking the inner product of the solution x values with a user-input array and comparing the result to a lower and upper bound.
#NonlinearConstraint: The solution is constrained by applying a user-supplied function to the solution x values and comparing the return value with a lower and upper bound.
#Bounds: The solution x values are constrained to lie between a lower and upper bound.

In [62]:
np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)

In [63]:
prices

array([0.77132064, 0.02075195, 0.63364823, 0.74880388, 0.49850701,
       0.22479665, 0.19806286, 0.76053071, 0.16911084, 0.08833981])

In [64]:
money_available

array([1, 1, 1, 3, 1, 3, 3, 2, 1, 1])

In [66]:
n_shares_per_buyer = money_available / prices
print(n_shares_per_buyer, sep="\n")

[ 1.29647768 48.18824404  1.57816269  4.00638948  2.00598984 13.34539487
 15.14670609  2.62974258  5.91328161 11.3199242 ]


In [67]:
constraint = LinearConstraint(np.ones(n_buyers), lb=n_shares, ub=n_shares)

In [76]:
np.ones(n_buyers)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [78]:
n_shares

15

In [68]:
constraint

In [69]:
bounds = [(0, n) for n in n_shares_per_buyer]

In [70]:
bounds

[(0, 1.296477682439221),
 (0, 48.18824403823818),
 (0, 1.5781626853523065),
 (0, 4.006389483224008),
 (0, 2.0059898362934296),
 (0, 13.3453948697305),
 (0, 15.146706090719757),
 (0, 2.629742583593113),
 (0, 5.913281610609325),
 (0, 11.31992419669592)]

In [71]:
def objective_function(x, prices):
    return -x.dot(prices)

In [72]:
res = minimize(
    objective_function,
    x0=10 * np.random.random(n_buyers),
    args=(prices,),
    constraints=constraint,
    bounds=bounds,
)

In [79]:
10 * np.random.random(n_buyers)

array([5.13138243, 6.50397182, 6.01038953, 8.05223197, 5.21647152,
       9.08648881, 3.19236089, 0.90459349, 3.00700057, 1.13984362])

In [73]:
res

     fun: -8.783020157087615
     jac: array([-0.7713207 , -0.02075195, -0.63364828, -0.74880385, -0.49850702,
       -0.22479653, -0.19806278, -0.76053071, -0.16911077, -0.08833981])
 message: 'Optimization terminated successfully'
    nfev: 187
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([1.29647768e+00, 1.07635367e-13, 1.57816269e+00, 4.00638948e+00,
       2.00598984e+00, 3.48323773e+00, 3.99680289e-15, 2.62974258e+00,
       1.94817649e-14, 2.79247930e-14])

In [74]:
print("The total number of shares is:", sum(res.x))
print("Leftover money for each buyer:", money_available - res.x * prices)

The total number of shares is: 15.000000000000002
Leftover money for each buyer: [1.37667655e-14 1.00000000e+00 1.17683641e-14 2.62012634e-14
 1.36557432e-14 2.21697984e+00 3.00000000e+00 2.30926389e-14
 1.00000000e+00 1.00000000e+00]
